In [ ]:
# Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
%cd Mecab-ko-for-Google-Colab

In [ ]:
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

## 1. 데이터 로드

In [ ]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk

In [ ]:
# 네이버 쇼핑 리뷰
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 : ', len(total_data))

In [ ]:
total_data.head()

## hgtk 튜토리얼

word embedding -> 단어 단위 임베딩
character embeding -> 문자 단위 임베딩


한국어를 문자단위로 임베딩 할 수 있게 하는것 -> 자음모음 분리기 hgtk

In [ ]:
# 한글인지 체크
print(hgtk.checker.is_hangul('ㄱ'))
print(hgtk.checker.is_hangul('12'))
print(hgtk.checker.is_hangul('a'))

In [ ]:
# 음절을 초성, 중성, 종성으로 나누기
print(hgtk.text.decompose('남'))

# 초성, 중성, 종성을 합치기
print(hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ'))

In [ ]:
# 결합할 수 없는 상황은 에러 발생
try:
  hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ') # 중성이 없는 경우
except:
  print('에러 발생')

## 3. 데이터 전처리

'fasttext' 는 subword 단위로 임베딩벡터를 생성해주는 도구
한국어는 subword 를 자음 모음 단위로 생각 가능

* fasttext에 학습시킬 데이터를 만들기 위해
* hgtk 를 통해 자음 모음 단위로 전처리 하자!


In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo): # 경우에 따라 초, 중, 종성이 다 있는 게 아닌 경우도 있음 이 경우 -를 반환해주는 함수
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [ ]:
print(word_to_jamo('남동생'))
print(word_to_jamo('야구')) # 야구의 경우 종성이 없으므로 종성 부분을 -로 반환

In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

In [ ]:
# mecab으로 형태소를 분리해주고 그 형태소마다 각각 자음모음을 분리해주는 함수
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

In [ ]:
# 리뷰 데이터의 reviews 컬럼만을 가져와서 자모 분리
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_numpy()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

In [ ]:
print(len(tokenized_data))
print("전처리 전:", total_data['reviews'][1])
print("전처리 후:", tokenized_data[1])

단어를 자모 분리한 상태를 다시 결합시키는 함수도 정의

-> 코사인 유사도 계산할때 단어 상태로 편하게 보기 위함

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

##4. Fasttext

In [ ]:
import fasttext

In [ ]:
with open('tokenized_data.txt', 'w') as out:   # 쓰기 모드로 전환
  for line in tqdm(tokenized_data, unit=' line'):     # 전처리된 데이터 입력
    out.write(' '.join(line) + '\n')

In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model("fasttext.bin")

In [ ]:
model = fasttext.load_model("fasttext.bin")

In [ ]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

get_nearest_neighbors 함수를 이용해 '남동생' 과 가장 유사한 단어를 k개 출력 (자모 분리)

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

가독성 좋게 출력

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('구매'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('배달'), k=10)))

## 5. Word2Vec

자모 단위가 아닌 단어단위의 임베딩벡터 생성

In [ ]:
# 간단하게 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

tokenized_data2 = []
for sentence in tqdm(total_data['reviews'].to_list()):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data2.append(stopwords_removed_sentence)

In [ ]:
print("word2vec용 데이터:", tokenized_data2[0])
print("fasttext용 데이터:", tokenized_data[0])

In [ ]:
from gensim.models import Word2Vec

model2 = Word2Vec(sentences = tokenized_data2, vector_size = 1000, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
model2.wv.vectors.shape

## 6. FastText 와 Word2Vec 결과 비교

In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('손수건'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("손수건"))

In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('판매'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("판매"))

## Comment

Fasttext 가 유사도 수치상 높은 결과를 보임

* Fasttext 는 자모음의 '형태'를 기준으로
* Word2Vec는 단어의 '의미'를 기준으로 우선출력